<a href="https://colab.research.google.com/github/shivengg87/AI-ML/blob/main/ANNwithpytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# PyTorch core
import torch   #tensor math
import torch.nn as nn # neural network layers
import torch.optim as optim # gradient descent logic

# Data utilities -> real dataset + preprocessing
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import numpy as np


In [ ]:
#  Load Data set
housing = fetch_california_housing()
